把lepton storage里的danbooru dataset准备成tfrecord格式，然后用tf.data.Dataset读取:

## get dataset paths:

(copied from https://github.com/arot-devs/proclib/blob/feat-anime-aesthetic-cls/notebooks/split_and_retrieve_files.ipynb)

In [1]:
import os
import random
import pandas as pd

import unibox as ub


CONFIG = {
    "twitter":{
        "img_root_dirs": ["/lv0/gallery-dl/twitter"],
        "chunks_dir": "/lv0/test_aesthetics/proclib/data/twitter_path_chunks"
    },
    "danbooru":{
        "img_root_dirs": [
            "/rmt/nyanko/danbooru2023-files", 
            "/rmt/nyanko/danbooru-extend",
            "/rmt/nyanko/danbooru-extend-2"
            ],
        "chunks_dir": "/lv0/test_aesthetics/proclib/data/danbooru_path_chunks"
    }
}

_curr_config = CONFIG["danbooru"]

IMG_ROOT_DIR = _curr_config["img_root_dirs"]
CHUNKS_DIR = _curr_config["chunks_dir"]

print(f"IMG_ROOT_DIR: {IMG_ROOT_DIR} \nCHUNKS_DIR: {CHUNKS_DIR}")

IMG_ROOT_DIR: ['/rmt/nyanko/danbooru2023-files', '/rmt/nyanko/danbooru-extend', '/rmt/nyanko/danbooru-extend-2'] 
CHUNKS_DIR: /lv0/test_aesthetics/proclib/data/danbooru_path_chunks


In [3]:
from proclib.utils import list_image_dirs_to_chunks

danbooru_chunk = list_image_dirs_to_chunks(IMG_ROOT_DIR, CHUNKS_DIR, num_chunks=1)

Processing image directories: ['/rmt/nyanko/danbooru2023-files', '/rmt/nyanko/danbooru-extend', '/rmt/nyanko/danbooru-extend-2'] -> /lv0/test_aesthetics/proclib/data/danbooru_path_chunks (1 chunks)


7538692 images found in /rmt/nyanko/danbooru2023-files | preview: ['/rmt/nyanko/danbooru2023-files/1029697.webp', '/rmt/nyanko/danbooru2023-files/1121188.webp', '/rmt/nyanko/danbooru2023-files/1334570.webp', '/rmt/nyanko/danbooru2023-files/1499385.webp', '/rmt/nyanko/danbooru2023-files/1520446.webp']


591816 images found in /rmt/nyanko/danbooru-extend | preview: ['/rmt/nyanko/danbooru-extend/7545225.webp', '/rmt/nyanko/danbooru-extend/7570711.webp', '/rmt/nyanko/danbooru-extend/7597977.webp', '/rmt/nyanko/danbooru-extend/7608653.webp', '/rmt/nyanko/danbooru-extend/7609228.webp']


4492 images found in /rmt/nyanko/danbooru-extend-2 | preview: ['/rmt/nyanko/danbooru-extend-2/image_8151508.jpg', '/rmt/nyanko/danbooru-extend-2/image_8151518.jpg', '/rmt/nyanko/danbooru-extend-2/image_8151521.jpg', '/rmt/nyanko/danbooru-extend-2/image_8151523.jpg', '/rmt/nyanko/danbooru-extend-2/image_8151570.jpg']
(8135000, 2)
Index(['local_path', 'filename'], dtype='object')


,local_path,filename
0,/rmt/nyanko/danbooru2023-files/1029697.webp,1029697.webp
1,/rmt/nyanko/danbooru2023-files/1121188.webp,1121188.webp
2,/rmt/nyanko/danbooru2023-files/1334570.webp,1334570.webp


Combined DataFrame preview: None


2024-12-16 06:34:31,472 [INFO] UniLogger: UniSaver.saves: DataFrame saved successfully to "/lv0/test_aesthetics/proclib/data/danbooru_path_chunks/0.parquet"


Chunking configuration saved to /lv0/test_aesthetics/proclib/data/danbooru_path_chunks/CHUNKING_INFO.txt


In [4]:
!mv /lv0/test_aesthetics/proclib/data/danbooru_path_chunks/0.parquet ./

In [6]:
!mv 0.parquet danbooru_paths.parquet

convert dataset:

In [9]:
import os

df = ub.loads("danbooru_paths.parquet")
df["path"] = df["local_path"]
df = df.drop(columns=["local_path"])
df["image_id"] = df["path"].apply(lambda x: os.path.basename(x).split(".")[0])

# remove "image_" prefix if exists
df["image_id"] = df["image_id"].apply(lambda x: x.split("_")[-1] if x.startswith("image_") else x)

df["image_id"] = df["image_id"].astype(int)

ub.peeks(df)

2024-12-16 06:41:52,019 [INFO] UniLogger: UniLoader.loads: .parquet LOADED from "danbooru_paths.parquet" in 6.23s


(8135000, 3)
Index(['filename', 'path', 'image_id'], dtype='object')


,filename,path,image_id
0,1029697.webp,/rmt/nyanko/danbooru2023-files/1029697.webp,1029697
1,1121188.webp,/rmt/nyanko/danbooru2023-files/1121188.webp,1121188
2,1334570.webp,/rmt/nyanko/danbooru2023-files/1334570.webp,1334570


In [13]:
print(f"Total images: {len(df)}")
print(f"unique danbooru image ids: {len(df['image_id'].unique())}")

print(f"first 5 duplicating ids: {df['image_id'].value_counts().head()}")

Total images: 8135000
unique danbooru image ids: 7938607
first 5 duplicating ids: image_id
7226424    2
7320412    2
7229500    2
7203284    2
7243353    2
Name: count, dtype: int64


In [15]:
df[df["image_id"] == 7320412]

,filename,path,image_id
5804953,7320412.webp,/rmt/nyanko/danbooru2023-files/7320412.webp,7320412
7396028,7320412.webp,/rmt/nyanko/danbooru2023-files/12/7320412.webp,7320412


In [16]:
# keep unique only
df = df.drop_duplicates(subset=["image_id"])
print(f"Total images: {len(df)}")
ub.saves(df, "danbooru_paths_unique.parquet")

Total images: 7938607


2024-12-16 06:45:22,580 [INFO] UniLogger: UniSaver.saves: DataFrame saved successfully to "danbooru_paths_unique.parquet"


## Adding Tag Info:

using data from aws version of danbooru to get info


TODO:
- dump latest danbooru meta
- upload to lepton
- extract tags
- concatenate to this df

In [ ]:
# TODO: get metadata



2024-12-16 06:48:49,611 [INFO] UniLogger: UniLoader.loads: .parquet LOADED from "/rmt/nyanko/danbooru_rev6_8_v7.parquet" in 17.67s


(3840716, 25)
Index(['is_yandere', 'created_at', 'tag_string', 'build_tags',
       'tag_string_character', 'tag_string_artist', 'tag_string_general',
       'file_path', 'clip_aesthetic', 'clip_aesthetic_2_5', 'phash',
       'laplacian_variance', 'width', 'height', 'artist_first', 'image_height',
       'image_width', 'is_special', 'is_lq_v2', 'is_lq', 'yandere_tags',
       'is_hq', 'is_tachie', 'is_lowres', 'is_hq_scenery'],
      dtype='object')


,is_yandere,created_at,tag_string,build_tags,tag_string_character,tag_string_artist,tag_string_general,file_path,clip_aesthetic,clip_aesthetic_2_5,...,image_height,image_width,is_special,is_lq_v2,is_lq,yandere_tags,is_hq,is_tachie,is_lowres,is_hq_scenery
1,False,2024-05-06T08:48:13.547-04:00,"[1girl, :p, alternate_costume, apron, blush, e...","[1girl, unconventional_maid, solo, maid_headdr...",[nanakusa_nichika],[morino_yu2],"[1girl, :p, alternate_costume, apron, blush, e...",/storage/training/nyanko/danbooru-extend/75455...,6.164867,5.947681,...,1753.0,1494.0,False,False,False,None,None,None,None,None
5,False,2024-05-14T00:36:56.332-04:00,"[1girl, absurdres, blue_eyes, blue_hair, blue_...","[1girl, blue_eyes, long_hair, solo, from_side,...",[],[di_le_to],"[1girl, blue_eyes, blue_hair, blue_ribbon, blu...",/storage/training/nyanko/danbooru-extend/75785...,5.700201,6.151587,...,2608.0,1608.0,False,False,False,None,None,None,None,None
6,False,2024-05-15T05:41:13.400-04:00,"[1girl, artist_name, ass, barefoot, blue_archi...","[1girl, panties, underwear, white_panties, hal...",[yuzu_(blue_archive)],[myeolchi],"[1girl, artist_name, ass, barefoot, blush, bow...",/storage/training/nyanko/danbooru-extend/75837...,5.368204,5.636465,...,1128.0,850.0,False,False,False,None,None,None,None,None
